In [1]:
# imports

import yaml

In [ ]:
# dynamically create intents

def add_intents_to_files(intents):
    # Open and read the existing NLU data from the nlu.yml file
    with open("data/nlu.yml", "r", encoding="utf-8") as nlu_file:
        nlu_data = yaml.load(nlu_file, Loader=yaml.FullLoader)

    # Open and read the existing domain data from the domain.yml file
    with open("domain.yml", "r", encoding="utf-8") as domain_file:
        domain_data = yaml.load(domain_file, Loader=yaml.FullLoader)

    # Add each intent to the NLU data and domain data
    for intent in intents:
        nlu_data["nlu"].append({"intent": intent["intent"], "examples": intent["examples"]})
        domain_data["intents"].append(intent["intent"])

    # Write the updated NLU data to the nlu.yml file
    with open("data/nlu.yml", "w", encoding="utf-8") as nlu_file:
        yaml.dump(nlu_data, nlu_file, allow_unicode=True, sort_keys=False)

    # Write the updated domain data to the domain.yml file
    with open("domain.yml", "w", encoding="utf-8") as domain_file:
        yaml.dump(domain_data, domain_file, allow_unicode=True, sort_keys=False)

In [ ]:
# call function: add_intents_to_files()

intents = [
    {
        "intent": "How_are_you",
        "examples": [
            "Como está?",
            "Tudo bem?",
            "Como estás?"
        ]
    }
]

add_intents_to_files(intents)

In [ ]:
# dynamically add more examples to intent

def add_examples_to_intent(intent_name, examples) -> None:

    with open("data/nlu.yml", "r", encoding="utf-8") as f:
        nlu = yaml.load(f, Loader=yaml.FullLoader)
        nlu_intents = nlu["nlu"]

        for intent in nlu_intents:
            if intent["intent"] == intent_name:
                intent["examples"].extend(examples)

    with open("data/nlu.yml", "w", encoding="utf-8") as f:
        yaml.dump(nlu, f, allow_unicode=True, sort_keys=False)

In [ ]:
# call function: add_examples_to_intent()

intent_name = "greet"
examples = ["hello", "hi", "hey"]
nlu_file = "./data/nlu.yml"

#add_examples_to_intent(intent_name, examples, nlu_file)

In [ ]:
# dynamically create intents with entities

def add_intents_with_entities_to_files(intents):
    # Open and read the existing NLU data from the nlu.yml file
    with open("data/nlu.yml", "r", encoding="utf-8") as nlu_file:
        nlu_data = yaml.load(nlu_file, Loader=yaml.FullLoader)

    # Open and read the existing domain data from the domain.yml file
    with open("domain.yml", "r", encoding="utf-8") as domain_file:
        domain_data = yaml.load(domain_file, Loader=yaml.FullLoader)
    
    # Add each intent to the NLU data and domain data
    
    for intent in intents:
        intent_dict = {"intent": intent["intent"], "examples": intent["examples"]}
        if intent.get("entities"):
            for entity in intent["entities"]:
                domain_data["entities"].append(entity)
                slot = {
                    "type":"text",
                    "mappings":[{"type":"from_entity","entity":entity}]
                    }
                domain_data["slots"][entity] = slot
        nlu_data["nlu"].append(intent_dict)
        domain_data["intents"].append(intent["intent"])

    # Write the updated NLU data to the nlu.yml file
    with open("data/nlu.yml", "w", encoding="utf-8") as nlu_file:
        yaml.dump(nlu_data, nlu_file, allow_unicode=True, sort_keys=False)

    # Write the updated domain data to the domain.yml file
    with open("domain.yml", "w", encoding="utf-8") as domain_file:
        yaml.dump(domain_data, domain_file, allow_unicode=True, sort_keys=False)

In [ ]:
# call function: add_intents_with_entities_to_files()
intents = [
    {
        "intent": "My_name",
        "examples": [
            "O meu nome é [Carla](user_name).",
            "Chamo-me [João](user_name).",
            "Eu sou a [Carla](user_name).",
            "Eu sou o [João](user_name).",
        ],
        "entities": ["user_name"]
    }
]

#add_intents_with_entities_to_files(intents)

In [ ]:
# create utterances and responses

def add_response(utterance: str, responses: str) -> None:

    with open("domain.yml", "r", encoding="utf-8") as f:
        domain =  yaml.load(f, Loader=yaml.FullLoader)
        
        # Add the response to the domain's responses
        if "responses" not in domain:
            domain["responses"] = {}

        domain["responses"][utterance] = [{"text": text} for text in responses]

    with open("domain.yml", "w", encoding="utf-8") as f:
       yaml.dump(domain, f, allow_unicode=True, sort_keys=False)

In [ ]:
# call function: add_response()

utterance_name = "utter_details_question"
response_texts = ["Por favor, explique o porquê.",
                  "Detalhe o porquê da sua resposta, por favor.",
                  "Por favor, explique a sua resposta.",
                  "Explique-me o motivo, por favor."]

add_response(utterance_name, response_texts)

In [11]:
# dynamically create rule

def add_custom_rule(rule_name: str, intent: str, actions: list, position: int):
    # Load existing rules from rules_file
    with open("data/rules.yml", 'r', encoding="utf-8") as f:
        rules = yaml.load(f, Loader=yaml.FullLoader)
    with open("domain.yml", "r", encoding="utf-8") as f:
        domain = yaml.load(f, Loader=yaml.FullLoader)
        intents = domain["intents"]
    if intent not in intents:
        print("Missing intent",intent)
        return 
    
    # Create the new rule
    rule = {"rule": rule_name, "steps": []}
    for i, action in enumerate(actions):
        typeAction = action.split("_")[0]
        if  typeAction == "utter" and "responses" in domain and action not in domain["responses"]:
            print("Missing utterance ", action)
            return
        if typeAction == "action" and "actions" in domain and action not in domain["actions"]:
            print("Missing action", action)
            return
        if i == position:
            rule["steps"].append({"intent": intent})
        rule["steps"].append({"action": action})

    # Add the new rule to the rules
    rules["rules"].append(rule)

    # Write updated rules to rules_file
    with open("data/rules.yml", 'w', encoding="utf-8") as f:
        yaml.dump(rules, f, allow_unicode=True, sort_keys=False)

In [16]:
# call function: add_custom_rule()
rule_name = "Yesavage depression scale - Q2 - Deny"
intent = "Deny_question"
actions = ["utter_question_2","action_add_point_questionnaire","utter_details_question", "utter_question_3"]
position = 1

add_custom_rule(rule_name, intent, actions, position)

In [13]:
# map a rule to a story

def create_story_from_rule(rule_name):

    # Load the rules.yml file
    with open("data/rules.yml", 'r', encoding="utf-8") as f:
        rules = yaml.load(f, Loader=yaml.FullLoader)

    with open("data/stories.yml", 'r', encoding="utf-8") as f:
        stories = yaml.load(f, Loader=yaml.FullLoader)

    # Find the specified rule
    rule = None
    for r in rules["rules"]:
        if r['rule'] == rule_name:
            rule = r
            break

    # If the rule was not found, raise an error
    if rule is None:
        return
    
    # check if story already exists
    for stor in stories["stories"]:
        if stor["story"] == rule_name:
            return
        
    # Create the story
    stories["stories"].append({"story": rule_name, "steps":rule['steps']})
    
    # Write the story to the stories.yml file
    with open("data/stories.yml", 'w', encoding="utf-8") as f:
         yaml.dump(stories, f, allow_unicode=True, sort_keys=False)

In [14]:
# call function: create_story_from_rule()

create_story_from_rule("Yesavage depression scale - Q1 - Affirm")

In [ ]:
# generate stories accordingly to rules

def verify_story(steps):
    # Load the rules from the file
    with open('data/rules.yml', 'r', encoding="utf-8") as f:
        rules = yaml.load(f, Loader=yaml.FullLoader)
        rules = rules["rules"]
    # Check for rule contradictions
    for rule in rules:
        ruleSteps = rule['steps']
        size = len(ruleSteps)-1
        for i, step in enumerate(ruleSteps):
            if step in steps:
                idxStep = steps.index(step)
                if i > 0:
                    previousStep = ruleSteps[i-1]
                    try:
                        idxPrev = steps.index(previousStep)
                        if previousStep in steps and idxStep < idxPrev:
                            return False
                    except ValueError:
                        print("Missing - "+previousStep[list(previousStep.keys())[0]]+" in story steps")
                        return False
                if i < size:
                    nextStep = ruleSteps[i+1]
                    try:
                        idxNext = steps.index(nextStep)
                        if nextStep in steps and idxStep > idxNext:
                            return False
                    except ValueError:
                        print("Missing - "+nextStep[list(nextStep.keys())[0]]+" in story steps")
                        return False
    return True

def create_story(story_name, stepsStory):

    if verify_story(stepsStory) == False:
        print("ERROR - rules")
        return         
    # Create the new story
    new_story = {'story': story_name, 'steps': stepsStory}

    # Load existing stories from the file
    with open('data/stories.yml', 'r', encoding="utf-8") as f:
        fileStories = yaml.load(f, Loader=yaml.FullLoader)
    # Check if the story already exists
    if any(s['story'] == story_name for s in fileStories["stories"]):
        print(f"Story '{story_name}' already exists in the stories file")
        return
    # Add the new story to the existing stories
    fileStories["stories"].append(new_story)

    # Write the updated stories to the file
    with open('data/stories.yml', 'w', encoding="utf-8") as f:
        yaml.dump(fileStories, f, allow_unicode=True, sort_keys=False)


In [ ]:
# call function: 

steps = []

story_name = "User says hello and asks features 1"
steps = [
    {"intent": "Mood_unhappy"},
    {"action": "utter_cheer_up"},
    {"action": "utter_did_that_help"}
]
#create_story(story_name, steps)

In [17]:
# Add action to domain.yml

def add_action_to_domain_file(action_name: str):
    with open("domain.yml", "r", encoding="utf-8") as f:
        domain = yaml.load(f, Loader=yaml.FullLoader)
    actions_index = len(domain["actions"])
    if action_name in domain["actions"]:
        return 
    domain["actions"].append(action_name)

    # Write the updated domain file
    with open("domain.yml", "w", encoding="utf-8") as f:
        yaml.dump(domain, f, allow_unicode=True, sort_keys=False)

In [3]:
# call function: add_action_to_domain_file() 

add_action_to_domain_file("action_collect_question_response")

In [8]:
# create slot
def create_slot_in_domain(slot_name, slot_data):
    """
    Creates a new slot with the given name and type, and adds it to the domain.yml file.
    """
    with open("domain.yml", "r", encoding="utf-8") as f:
        domain = yaml.load(f, Loader=yaml.FullLoader)

    # Create the new slot
    new_slot = {slot_name: slot_data}

    # Add the new slot to the domain file
    domain["slots"].update(new_slot)

    with open("domain.yml", "w", encoding="utf-8") as f:
        yaml.dump(domain, f, allow_unicode=True, sort_keys=False)

In [10]:
# call function create_slot_in_domain()

slot_name = "geriatric_questionnaire_points"
slot_data = {"type":"float","min_value":0.0, "max_value":15.0,"initial_value":0.0, "mappings": [{"type":"custom"}]}
create_slot_in_domain(slot_name,slot_data)

{'family_friends': {'type': 'text', 'mappings': [{'type': 'from_entity', 'entity': 'family_friends'}]}, 'user_name': {'type': 'text', 'mappings': [{'type': 'from_entity', 'entity': 'user_name'}]}}
